# Setup Colab Env

## Access ARQMath on Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
from os import listdir
from os.path import join
dirfiles = listdir("/content/drive/MyDrive/ARQMathAgg/dataset/")
print(dirfiles)

['collection_small.tsv', 'queries_small.tsv']


## Clone git repos

In [3]:
!git clone https://github.com/past12am/MathIR

fatal: destination path 'MathIR' already exists and is not an empty directory.


In [4]:
!git -Cl ColBERT/ pul || git clone https://github.com/stanford-futuredata/ColBERT.git
import sys; sys.path.insert(0, 'ColBERT/')

unknown option: -Cl
usage: git [--version] [--help] [-C <path>] [-c <name>=<value>]
           [--exec-path[=<path>]] [--html-path] [--man-path] [--info-path]
           [-p | --paginate | -P | --no-pager] [--no-replace-objects] [--bare]
           [--git-dir=<path>] [--work-tree=<path>] [--namespace=<name>]
           [--super-prefix=<path>] [--config-env=<name>=<envvar>]
           <command> [<args>]
fatal: destination path 'ColBERT' already exists and is not an empty directory.


### Setup ColBERT

In [5]:
try: # When on google Colab, let's install all dependencies with pip.
    import google.colab
    !pip install -U pip
    !pip install -e ColBERT/['faiss-gpu']
except Exception:
  import sys; sys.path.insert(0, 'ColBERT/')
  try:
    from colbert import Indexer, Searcher
  except Exception:
    print("If you're running outside Colab, please make sure you install ColBERT in conda following the instructions in our README. You can also install (as above) with pip but it may install slower or less stable faiss or torch dependencies. Conda is recommended.")
    assert False

Obtaining file:///content/ColBERT
  Preparing metadata (setup.py) ... done
  Attempting uninstall: colbert-ai
    Found existing installation: colbert-ai 0.2.20
    Uninstalling colbert-ai-0.2.20:
      Successfully uninstalled colbert-ai-0.2.20
  DEPRECATION: Legacy editable install of colbert-ai[faiss-gpu]==0.2.20 from file:///content/ColBERT (setup.py develop) is deprecated. pip 25.0 will enforce this behaviour change. A possible replacement is to add a pyproject.toml or enable --use-pep517, and use setuptools >= 64. If the resulting installation is not behaving as expected, try using --config-settings editable_mode=compat. Please consult the setuptools documentation for more information. Discussion can be found at https://github.com/pypa/pip/issues/11457
  Running setup.py develop for colbert-ai


# Index Dataset with ColBERT

In [6]:
import os
import colbert

from colbert import Indexer, Searcher
from colbert.infra import Run, RunConfig, ColBERTConfig
from colbert.data import Queries, Collection

In [8]:
dataroot = '/content/drive/MyDrive/ARQMathAgg/dataset/'
collection = os.path.join(dataroot, 'collection_small.tsv')


nbits = 4   # encode each dimension with 2 bits
doc_maxlen = 500   # truncate passages at 300 tokens

checkpoint = 'colbert-ir/colbertv2.0'
index_name = f'arqmath.full.{nbits}bits'

with Run().context(RunConfig(nranks=1, root="/content/drive/MyDrive/ARQMathAgg/ColBERTIdxs/", experiment="ColBERTTestARQMath")):

    config = ColBERTConfig(
        nbits=nbits,
        doc_maxlen=doc_maxlen,
        kmeans_niters=10
    )


    print("Indexing")
    indexer = Indexer(checkpoint=checkpoint, config=config)
    indexer.index(name=index_name, collection=collection, overwrite=True)

Indexing


[Dec 26, 17:23:30] #> Creating directory /content/drive/MyDrive/ARQMathAgg/ColBERTIdxs/ColBERTTestARQMath/indexes/arqmath.full.4bits 


#> Starting...
#> Joined...
